# Fake News Prediction using logistic regression 

id: represent the unique value of the article
title: title of the news aticle
author: author of the news aticle
text: the text of the article, could be incomplete
label: A label that mark if the news article is real or fake

real article : 0
fake article : 1

# Importing the libraries

In [1]:
import numpy as  np
import pandas as pd
import re #library useful for searching text in the document
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer #useful in converting text into binary for our machine learning algo to read properly
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fahd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
print(stopwords.words('english')) #This are the words to be removed from our dataset during the stemming process as they dont add much value to our data

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# Data Pre-processing

In [4]:
#Loading the dataset  to the pandas dataframe
pd.read_csv('submit.csv')
news_dataset = pd.read_csv('train.csv')
pd.read_csv('test.csv')

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...
...,...,...,...,...
5195,25995,The Bangladeshi Traffic Jam That Never Ends - ...,Jody Rosen,Of all the dysfunctions that plague the world’...
5196,25996,John Kasich Signs One Abortion Bill in Ohio bu...,Sheryl Gay Stolberg,WASHINGTON — Gov. John Kasich of Ohio on Tu...
5197,25997,"California Today: What, Exactly, Is in Your Su...",Mike McPhate,Good morning. (Want to get California Today by...
5198,25998,300 US Marines To Be Deployed To Russian Borde...,NaN,« Previous - Next » 300 US Marines To Be Deplo...


In [5]:
news_dataset.shape

(20800, 5)

In [6]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
#To check if sum values are missing in the dataset
print(news_dataset.isnull().sum())

id           0
title      558
author    1957
text        39
label        0
dtype: int64


In [8]:
news_dataset[news_dataset.label == 1]

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
...,...,...,...,...,...
20788,20788,Maine’s Gov. LePage Threatens To ‘Investigate’...,Joe Clark,Google Pinterest Digg Linkedin Reddit Stumbleu...,1
20791,20791,Lawyer Who Kept Hillary Campaign Chief Out of ...,Daniel Greenfield,Lawyer Who Kept Hillary Campaign Chief Out of ...,1
20793,20793,Idiot Who Destroyed Trump Hollywood Star Gets ...,Robert Rich,Share This \nAlthough the vandal who thought i...,1
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [9]:
news_dataset = news_dataset.fillna('')# replacing the null value with empty dtring

In [10]:
#let check how the data loooks like again after filling with nas
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [11]:
news_dataset.iloc[6]

id                                                        6
title     Life: Life Of Luxury: Elton John’s 6 Favorite ...
author                                                     
text      Ever wonder how Britain’s most iconic pop pian...
label                                                     1
Name: 6, dtype: object

In [12]:
#Merging the author name and news title
news_dataset['content'] = news_dataset['author'] +' '+ news_dataset['title']

In [13]:
news_dataset['content'].iloc[0]

'Darrell Lucus House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [14]:
news_dataset.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [15]:
X = news_dataset.drop(['label'], axis = 1)
Y = news_dataset['label']

In [16]:
X.head()
Y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [17]:
#Stemming examples are acting, actor, actress -- (ACT)

# Stemming is the process  of reducing a word to it root word

In [18]:
port_stem = PorterStemmer()

In [19]:
def stemming(contentt):
    stemmed_content = re.sub('[^a-zA-Z]',' ',str(contentt))#this line of code is saying to search and subtitute which is the usefullness of the function re.sub except (^) a-zA-Z with space in the parameters contentt
    stemmed_content = stemmed_content.lower()#to make ever alphabet a lowercase
    stemmed_content = stemmed_content.split()#to split every word properly and made each word a list
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] #the looping function is to loop all the phrase in the variable stemmed_content and pass it to word excluding the phrases in stop.words after then passing it to port_stem.words() to numalize each words 
    stemmed_content = ' '.join(stemmed_content)# after we are joining all the words
    return stemmed_content                                                                                                  
                                                                                                       

In [20]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [21]:
news_dataset = news_dataset.drop(['ontent'], axis = 1)

KeyError: "['ontent'] not found in axis"

In [22]:
news_dataset

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,darrel lucu hous dem aid even see comey letter...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,daniel j flynn flynn hillari clinton big woman...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,consortiumnew com truth might get fire
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,jessica purkiss civilian kill singl us airstri...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,howard portnoy iranian woman jail fiction unpu...
...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,jerom hudson rapper trump poster child white s...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,benjamin hoffman n f l playoff schedul matchup...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,michael j de la merc rachel abram maci said re...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,alex ansari nato russia hold parallel exercis ...


In [49]:
news_dataset['content']

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object

In [157]:
J = news_dataset['content']


In [158]:
J

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object

In [156]:
Y = news_dataset['label']
Y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64

In [82]:
#Converting the textual data to numerical data for the computer to understand

In [159]:
vectorizer = TfidfVectorizer()

vectorizer.fit(J)

J = vectorizer.transform(J)

In [160]:
print(J)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

In [161]:
J.shape

(20800, 17128)

# Fitting the dataset into training and testing data

In [162]:
J_train, J_test, Y_train, Y_test = train_test_split(J, Y, test_size = 0.2, stratify = Y, random_state = 2)

# Training the model : Logistic regression model

In [163]:
model = LogisticRegression()

In [164]:
ml = model.fit(J_train, Y_train)

# Evaluation

In [181]:
#Accuracy score on training data
J_train_prediction = model.predict(J_train)

In [183]:
J_training_data_accuracy = accuracy_score(J_train_prediction, Y_train)

In [184]:
J_training_data_accuracy

0.9865985576923076

In [186]:
#Accuracy score on testing data
J_testing_prediction = model.predict(J_test)

In [187]:
J_testing_data_accuracy = accuracy_score(J_testing_prediction,  Y_test)

In [188]:
J_testing_data_accuracy

0.9790865384615385

# Making a predictive system

In [202]:
x_new = J_test[0]
print(x_new)

  (0, 12801)	0.2910746804557067
  (0, 9818)	0.30786004182651133
  (0, 7668)	0.22945314906455008
  (0, 6816)	0.16094563145945953
  (0, 6289)	0.288254092437116
  (0, 5941)	0.288254092437116
  (0, 5233)	0.21316265672448448
  (0, 4346)	0.3250084367199054
  (0, 3395)	0.3301936745912874
  (0, 2959)	0.24534646237198773
  (0, 1667)	0.30373060380734146
  (0, 908)	0.213510750423647
  (0, 239)	0.34297808354766485


In [192]:
prediction = model.predict(x_new)

In [193]:
print(prediction)

[1]


In [195]:
print(Y_test[0])

1


In [197]:
if prediction[0] == 0:
    print('The news is real')
else:
    print('The news is fake')


The news is fake


In [237]:
x_news = J_test[1]
print(x_news)

  (0, 16996)	0.09117761343372983
  (0, 15295)	0.08946281236254729
  (0, 14046)	0.42524648908354634
  (0, 13190)	0.36773046084789346
  (0, 12741)	0.24868518461414146
  (0, 12279)	0.3796661151115819
  (0, 12041)	0.37327055071909065
  (0, 10306)	0.08813410128297053
  (0, 8813)	0.42524648908354634
  (0, 4008)	0.23098933893199997
  (0, 3339)	0.2834482751186189


In [238]:
predictions = model.predict(x_news)
print(predictions)

if predictions == 0:
    print('the news is real')
else:
    print('the news is fake')

[0]
the news is real


In [243]:
print(Y_test[0])

1


In [242]:
Y_test[0]

1

In [229]:
print(J_test)

  (0, 12801)	0.2910746804557067
  (0, 9818)	0.30786004182651133
  (0, 7668)	0.22945314906455008
  (0, 6816)	0.16094563145945953
  (0, 6289)	0.288254092437116
  (0, 5941)	0.288254092437116
  (0, 5233)	0.21316265672448448
  (0, 4346)	0.3250084367199054
  (0, 3395)	0.3301936745912874
  (0, 2959)	0.24534646237198773
  (0, 1667)	0.30373060380734146
  (0, 908)	0.213510750423647
  (0, 239)	0.34297808354766485
  (1, 16996)	0.09117761343372983
  (1, 15295)	0.08946281236254729
  (1, 14046)	0.42524648908354634
  (1, 13190)	0.36773046084789346
  (1, 12741)	0.24868518461414146
  (1, 12279)	0.3796661151115819
  (1, 12041)	0.37327055071909065
  (1, 10306)	0.08813410128297053
  (1, 8813)	0.42524648908354634
  (1, 4008)	0.23098933893199997
  (1, 3339)	0.2834482751186189
  (2, 16868)	0.344315415802567
  :	:
  (4158, 16782)	0.2796201559111399
  (4158, 13954)	0.2713157796360236
  (4158, 13918)	0.29432702492466434
  (4158, 13775)	0.21479309786895925
  (4158, 13600)	0.2496550139728973
  (4158, 11068)	0.2943